In [2]:
import os
from pathlib import Path
import requests
import zipfile
import tarfile
import shutil
from warnings import filterwarnings
filterwarnings('ignore')
def download_to_cache(url, filename, force_download=False):
    def _get_custom_cache_dir():
        custom_cache_dir = Path.home() / ".cache"/ "skmini"/ "datasets"
        os.makedirs(custom_cache_dir, exist_ok=True)
        return custom_cache_dir

    file_path = _get_custom_cache_dir() / filename
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        if not force_download:
            print('File Aldready exists. Use force_download=True to force download.')
    else:
        force_download=True

    if force_download:
        try:
            if os.path.isfile(file_path) or os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove the directory and all its contents
                print(f"Successfully deleted the directory and its contents: {file_path}")
        except OSError as e:
            print(f"Error: {e.strerror}")

        response = requests.get(url, stream=True, verify=False)
        print(response)
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
        print(f"Downloaded file to {file_path}")
        # now that it is downloaded, we might want to decompress the file
        if tarfile.is_tarfile(file_path):
            try:
                tar = tarfile.open(file_path)
                os.remove(file_path)
                tar.extractall(path = file_path)
                tar.close()
            except:
                print('Decompression Failed.')
        elif zipfile.is_zipfile(file_path): # change to endswith, which doesnt require us loading a library
            try:
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    print(file_path)
                    os.remove(file_path)
                    os.mkdir(file_path)
                    zip_ref.extractall(file_path)
            except: print('Decompression Failed.')


In [3]:
from sklearn import datasets
x = datasets.load_digits()
# print(x.shape)
x
# x.reshape((-1,28,28))

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(x.images[10])

In [ ]:
dir(datasets)

In [ ]:
x = [0,0,5,13,9,1,0,0,0,0,13,15,10,15,5,0,0,3,15,2,0,11,8,0,0,4,12,0,0,8,8,0,0,5,8,0,0,9,8,0,0,4,11,0,1,12,7,0,0,2,14,5,10,12,0,0,0,0,6,13,10,0,0,0,0]
len(x)

In [ ]:
from skmini.datasets import load_cifar10, load_iris, load_digits, load_diabetes, load_squad, load_imdb
data = load_imdb(force_download=False)

In [ ]:
data['data'][15003], data['target'][15003]

In [ ]:
from skmini.classification import KNN
from skmini.datasets import load_iris
import numpy as np

In [ ]:
X = load_iris()['data']
y = load_iris()['target']
X.shape, y.shape
model = KNN(2)
model.fit(X,y)

In [ ]:
model.score(X, y)

In [ ]:
model.predict(X) == y

# SVM

In [ ]:
from sklearn.svm import SVCClassifier

In [ ]:
import sklearn
dir(sklearn.svm.SVC)

In [ ]:
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.svm import SVC

# Load the datasets
cancer = load_breast_cancer()
X = cancer.data[:, :2]
y = cancer.target

#Build the model
svm = SVC(kernel="rbf", gamma=0.5, C=1.0)
# Trained the model
svm.fit(X, y)

# Plot Decision Boundary
DecisionBoundaryDisplay.from_estimator(
        svm,
        X,
        response_method="predict",
        cmap=plt.cm.Spectral,
        alpha=0.8,
        xlabel=cancer.feature_names[0],
        ylabel=cancer.feature_names[1],
    )

# Scatter plot
plt.scatter(X[:, 0], X[:, 1], 
            c=y, 
            s=20, edgecolors="k")
plt.show()

In [1]:
from skmini.datasets import load_iris
from skmini.classification import SVC
# X = load_iris()['data']
# y = load_iris()['target']
# print(X.shape, y.shape)
import numpy as np
import matplotlib.pyplot as plt

X = np.array([[3,4,5],[5,4,2],[13,6,4],[17, 6, 3]])
y = np.array([0, 0, 1, 1])
svm = SVC()
svm.fit(X, y)

1.0061593350986073
0.5973059121929234
2.7905673430795703
0.37030941194804645
1.8264463907087618
1.8572820112295836
0
0
0.9362434135170736
1.016868392940961
1.4722314369621587
0
0.8666023802875493
1.1367549378752326
1.0529874515532334
0
0.7970250407733089
1.2565535999428117
0.6340707631271738
0
0.7275113440346649
1.376264449428902
0.21548110992493386
0
0.6580612391726692
1.4958875565624972
0
0
0
1.0558109527164232
0.7581221283846076
0
0.18988132974872363
1.1757063409494424
0.3394092710575609
0
0.12088519268511655
1.2955138392432526
0
0
